In [1]:
import mariadb
import pandas as pd
import json
from datetime import datetime, timedelta


# Conectar ao banco de dados
conexao = mariadb.connect(
    host="54.196.208.38",
    user="root",
    password="qiykGUao3R.D",
    database="IDPB",
    port=3306
)

# Criar um cursor
sql = conexao.cursor()

# Executar a consulta
sql.execute("SELECT * FROM membros")
resultados = sql.fetchall()

# Obter a descrição das colunas
colunas = [desc[0] for desc in sql.description]


# Criar um DataFrame com os resultados
df_membros = pd.DataFrame(resultados, columns=colunas)

# Salvar o df em csv
df_membros.to_csv('dados_csv/dados_membros.csv', index=False)

# Exibir o DataFrame
#df_membros

In [2]:
# Executar a consulta
sql.execute("SELECT * FROM celulas")
resultados = sql.fetchall()

# Obter a descrição das colunas
colunas = [desc[0] for desc in sql.description]

# Criar um DataFrame com os resultados
df_celulas = pd.DataFrame(resultados, columns=colunas)

# Salvar o df em csv
df_celulas.to_csv('dados_csv/dados_celulas.csv', index=False)

# Exibir o DataFrame
#df_celulas.head()


In [3]:
# Executar a consulta
sql.execute("SELECT * FROM usuarios")
resultados = sql.fetchall()

# Obter a descrição das colunas
colunas = [desc[0] for desc in sql.description]

# Criar um DataFrame com os resultados
df_usuarios = pd.DataFrame(resultados, columns=colunas)

# Salvar o df em csv
df_usuarios.to_csv('dados_csv/dados_usuarios.csv', index=False)

# Exibir o DataFrame
#df_usuarios

In [4]:
# Fechar o cursor e a conexão
sql.close()
conexao.close()

In [5]:
# Contagem de membros
total_membros = df_membros['cpf'].nunique()

# Contagem de profissões
todas_profissoes = df_membros['profissao']
todas_profissoes = todas_profissoes.tolist()
#todas_profissoes

# Calcular as idades
current_year = datetime.now().year
df_membros['data_nascimento'] = pd.to_datetime(df_membros['data_nascimento'], format='%Y-%m-%d')
df_membros['idade'] = current_year - df_membros['data_nascimento'].dt.year

# Calcular total de crianças (até 12 anos)
total_criancas = df_membros[df_membros['idade'] <= 12].shape[0]

# Calcular total de jovens (13 até 29 anos)
total_jovens = df_membros[(df_membros['idade'] >= 13) & (df_membros['idade'] <= 29)].shape[0]

# Calcular total de idosos (+60)
total_idosos = df_membros[df_membros['idade'] >= 60].shape[0]

# Calcular totais de relacionamentos
totais_relacionamentos = df_membros['estado_civil'].value_counts().to_dict()

# Calcular totais de escolaridade
totais_escolaridade = df_membros['escolaridade'].value_counts().to_dict()

# Calcular o total de homens acima de 12 anos
total_homens = df_membros[(df_membros['idade'] > 12) & (df_membros['genero'] == 'Masculino')].shape[0]

# Calcular total mulheres acima de 12 anos
total_mulheres = df_membros[(df_membros['idade'] > 12) & (df_membros['genero'] == 'Feminino')].shape[0]

# Calcular contagem pelo número da célula
contagem_por_celula = df_membros['numero_celula'].value_counts().to_dict()

# Total de usuários do sistema
total_usuarios = df_usuarios['id_Usuario'].shape[0]
#total_usuarios

'''
# Exibir os resultados
print(f'Total de Pessoas: {total_membros}')
print(f'Total de Crianças (até 12 anos): {total_criancas}')
print(f'Total de Jovens (13 até 29 anos): {total_jovens}')
print(f'Total de Idosos (+60): {total_idosos}')
print(f'Totais de Relacionamentos: {totais_relacionamentos}')
print(f'Totais de Escolaridade: {totais_escolaridade}')
print(f'Totais de Homens: {total_homens}')
print(f'Totais de Mulheres: {total_mulheres}')
print(f'Contagem por Célula: {contagem_por_celula}')
print(f'Profissões: {todas_profissoes}')
print(f'Usuários do sistema: {total_usuarios}')
'''


"\n# Exibir os resultados\nprint(f'Total de Pessoas: {total_membros}')\nprint(f'Total de Crianças (até 12 anos): {total_criancas}')\nprint(f'Total de Jovens (13 até 29 anos): {total_jovens}')\nprint(f'Total de Idosos (+60): {total_idosos}')\nprint(f'Totais de Relacionamentos: {totais_relacionamentos}')\nprint(f'Totais de Escolaridade: {totais_escolaridade}')\nprint(f'Totais de Homens: {total_homens}')\nprint(f'Totais de Mulheres: {total_mulheres}')\nprint(f'Contagem por Célula: {contagem_por_celula}')\nprint(f'Profissões: {todas_profissoes}')\nprint(f'Usuários do sistema: {total_usuarios}')\n"

In [6]:
# Garantir que os tipos de dados das colunas sejam consistentes
df_membros['numero_celula'] = df_membros['numero_celula'].astype(int)
df_celulas['numero_celula'] = df_celulas['numero_celula'].astype(int)

# Mesclar df_membros com df_celulas para obter supervisão e coordenação dos membros
df_membros_celulas = pd.merge(df_membros, df_celulas, on='numero_celula')

# Contagem de células por supervisão
contagem_celulas_supervisao = df_celulas.groupby('numero_supervisao')['numero_celula'].count().reset_index()
contagem_celulas_supervisao.columns = ['numero_supervisao', 'contagem_celulas']

# Contagem de membros por supervisão
contagem_membros_supervisao = df_membros_celulas.groupby('numero_supervisao')['cpf'].count().reset_index()
contagem_membros_supervisao.columns = ['numero_supervisao', 'contagem_membros']

# Mesclar contagens de células e membros por supervisão
contagem_supervisao = pd.merge(contagem_celulas_supervisao, contagem_membros_supervisao, on='numero_supervisao')

# Contagem de células por coordenação
contagem_celulas_coordenacao = df_celulas.groupby('numero_coordenacao')['numero_celula'].count().reset_index()
contagem_celulas_coordenacao.columns = ['numero_coordenacao', 'contagem_celulas']

# Contagem de membros por coordenação
contagem_membros_coordenacao = df_membros_celulas.groupby('numero_coordenacao')['cpf'].count().reset_index()
contagem_membros_coordenacao.columns = ['numero_coordenacao', 'contagem_membros']

# Mesclar contagens de células e membros por coordenação
contagem_coordenacao = pd.merge(contagem_celulas_coordenacao, contagem_membros_coordenacao, on='numero_coordenacao')

In [7]:
# Transformar para dicionário / Coordenação
contagem_coordenacao = contagem_coordenacao.set_index('numero_coordenacao').to_dict(orient='index')
contagem_coordenacao = {k: {'contagem_celulas': v['contagem_celulas'], 'contagem_membros': v['contagem_membros']} for k, v in contagem_coordenacao.items()}

# Transformar para dicionário / Supervisão
contagem_supervisao = contagem_supervisao.set_index('numero_supervisao').to_dict(orient='index')
contagem_supervisao = {k: {'contagem_celulas': v['contagem_celulas'], 'contagem_membros': v['contagem_membros']} for k, v in contagem_supervisao.items()}

In [8]:
# Converte a coluna de data de nascimento para o formato de data
df_membros['data_nascimento'] = pd.to_datetime(df_membros['data_nascimento'])

# Obtém o mês e ano atual
hoje = datetime.now()
mes_atual = hoje.month

# Adiciona colunas para o mês de nascimento e o ano de nascimento
df_membros['mes_nascimento'] = df_membros['data_nascimento'].dt.month

# Filtra os aniversariantes do mês atual
aniversariantes_mes_atual = df_membros[df_membros['mes_nascimento'] == mes_atual]

# Cria o dicionário com a data de aniversário como chave e um dicionário com nome e CPF como valor
aniversariantes_mes = {}
for index, row in aniversariantes_mes_atual.iterrows():
    data_nascimento = row['data_nascimento'].strftime('%Y-%m-%d')
    aniversariantes_mes[data_nascimento] = {
        'nome_completo': row['nome_completo'],
        'cpf': row['cpf']
    }

In [9]:
# Converte a coluna de data de conversão para o formato de data
df_membros['data_conversao'] = pd.to_datetime(df_membros['data_conversao'], errors='coerce')

# Cria o array com nome do membro e data de conversão
total_convertidos = []
for index, row in df_membros.iterrows():
    nome = row['nome_completo']
    data_conversao = row['data_conversao'].strftime('%Y-%m-%d') if pd.notna(row['data_conversao']) else None
    total_convertidos.append({
        'nome_completo': nome,
        'data_conversao': data_conversao
    })



In [10]:
# Obtém a data atual e a data de um ano atrás
hoje = datetime.now()
um_ano_atras = hoje - timedelta(days=365)

# Filtra os novos convertidos (convertidos nos últimos 12 meses)
novos_convertidos_df = df_membros[(df_membros['data_conversao'] >= um_ano_atras) & (df_membros['data_conversao'] <= hoje)]

novos_convertidos = []
for index, row in novos_convertidos_df.iterrows():
    nome = row['nome_completo']
    data_conversao = row['data_conversao'].strftime('%Y-%m-%d') if pd.notna(row['data_conversao']) else None
    novos_convertidos.append({
        'nome_completo': nome,
        'data_conversao': data_conversao
    })

# Imprime o resultado
#print(novos_convertidos)

In [11]:
# Converte a coluna de data de conversão para o formato de data
df_membros['data_batismo'] = pd.to_datetime(df_membros['data_batismo'], errors='coerce')

# Cria o array com nome do membro e data de conversão
total_batizados = []
for index, row in df_membros.iterrows():
    nome = row['nome_completo']
    data_batismo = row['data_batismo'].strftime('%Y-%m-%d') if pd.notna(row['data_batismo']) else None
    total_batizados.append({
        'nome_completo': nome,
        'data_batismo': data_batismo
    })

# Imprime o resultado
#print(total_batizados)

In [12]:
# Total de participantes de ministério
total_participantes_ministerio = len(df_membros[df_membros['participacao_ministerio'].notna()])

# Criação do array com nome, CPF, e-mail e ministério
participantes_ministerio = []
for index, row in df_membros.iterrows():
    participantes_ministerio.append({
        'nome_completo': row['nome_completo'],
        'cpf': row['cpf'],
        'email': row['email'],
        'participacao_ministerio': row['participacao_ministerio']
    })



In [15]:
# Criar dicionário e salvar em JSON
dados_json = {
    "total_membros": total_membros,
    "total_criancas": total_criancas,
    "total_jovens": total_jovens,
    "total_idosos": total_idosos,
    "totais_relacionamentos": totais_relacionamentos,
    "totais_escolaridade":totais_escolaridade,
    "total_homens": total_homens,
    "total_mulheres": total_mulheres,
    "contagem_por_celula": contagem_por_celula,
    "contagem_supervisao": contagem_supervisao,
    "contagem_coordenacao": contagem_coordenacao,
    "total_usuarios": total_usuarios,
    "aniversariantes_mes": aniversariantes_mes
}



In [18]:
participa_ministerio = {
    "participantes_ministerio": participantes_ministerio
}

batizados = {
    "total_batizados": total_batizados
}

convertidos = {
    "total_convertidos": total_convertidos
}

convertidos_novos = {
    "novos_convertidos": novos_convertidos
}

profissoes = {
    "profissoes": todas_profissoes
}


In [19]:
# Salvar em arquivos JSON
with open('json/dados_analisados.json', 'w', encoding='utf-8') as f:
    json.dump(dados_json, f, ensure_ascii=False, indent=4)

# Participantes ou não de ministérios
with open('json/participa_ministerio.json', 'w', encoding='utf-8') as f:
    json.dump(participa_ministerio, f, ensure_ascii=False, indent=4)

# Batizados
with open('json/batizados.json', 'w', encoding='utf-8') as f:
    json.dump(batizados, f, ensure_ascii=False, indent=4)

# Convertidos
with open('json/convertidos.json', 'w', encoding='utf-8') as f:
    json.dump(convertidos, f, ensure_ascii=False, indent=4)
    
# Novos Convertidos
with open('json/convertidos_novos.json', 'w', encoding='utf-8') as f:
    json.dump(convertidos_novos, f, ensure_ascii=False, indent=4)

# Profissoes
with open('json/profissoes.json', 'w', encoding='utf-8') as f:
    json.dump(profissoes, f, ensure_ascii=False, indent=4)